##        Artificial Neural Network for Classification
Used to solve a data analytics challenge for a bank looking to determine consumer credit risk.

This is an Artificial Neural Network that can predict, based on 24 attributes of
a customer, if an individual customer will default on their payment next month for their credit card.

In addition, we are be able to rank all the customers of the bank, based on 
their probability of default. To do that, we use the right Deep Learning model, 
one that is based on a probabilistic approach (need the output layer(dependent variable)
to use the sigmoid activation function

In [1]:
# ===========================
# Data Preprocessing
# ===========================

# Importing the libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
'''
Dataset Information:
    Data Set Characteristics: Multivariate
    Attribute Characteristics: Integer, Real
    Number of Attributes: 24
    Number of Instances: 30000
    Source: 
        UCI Machine Learning Repository
        institutions: (1) Department of Information Management, Chung Hua University, Taiwan. (2) Department of Civil Engineering, Tamkang University, Taiwan. 
    
Dataset Attributes:
    This research employed a binary variable, default payment (Yes = 1, No = 0), as the response variable. 
    This study reviewed the literature and used the following 23 variables as explanatory variables: 
        
    X1: Amount of the given credit (NT dollar): it includes both the individual consumer credit and his/her family (supplementary) credit. 
    X2: Gender (1 = male; 2 = female). 
    X3: Education (1 = graduate school; 2 = university; 3 = high school; 4 = others). 
    X4: Marital status (1 = married; 2 = single; 3 = others). 
    X5: Age (year). 
    X6 - X11: History of past payment. We tracked the past monthly payment records (from April to September, 2005) as follows: X6 = the repayment status in September, 2005; X7 = the repayment status in August, 2005; . . .;X11 = the repayment status in April, 2005. The measurement scale for the repayment status is: -1 = pay duly(paid appropriately); 1 = payment delay for one month; 2 = payment delay for two months; . . .; 8 = payment delay for eight months; 9 = payment delay for nine months and above. 
    X12-X17: Amount of bill statement (NT dollar). X12 = amount of bill statement in September, 2005; X13 = amount of bill statement in August, 2005; . . .; X17 = amount of bill statement in April, 2005. 
    X18-X23: Amount of previous payment (NT dollar). X18 = amount paid in September, 2005; X19 = amount paid in August, 2005; . . .;X23 = amount paid in April, 2005.
'''
# Import the dataset
# Note we have two headers in dataset, skip the extra row
dataset = pd.read_excel('./Downloads/default-of-credit-card-clients.xls', skiprows=1)



In [3]:
# In[2]:

# matrix of features (independent variables). 
# select: all rows. columnn indices 1 - 23 (exclude columns: id#)
X = dataset.iloc[:, 1:24].values 

# matrix of dependent vars (output to predict)
# select: all rows. last column
y = dataset.iloc[:, -1].values


In [30]:
# In[3]:
# Encode categorical data (converting into numbers to make it comparable for 
# Regression) and create dummy vars if needed(avoid dummy var trap)

# This step is not needed since our dataset has no categorical variables


# In[4]:

# Split the dataset into the Training set and Test set
# Train:  0.8 (80%) of obesrvations to train ANN.
# Test: 0.2 (20%) of obesrvations to test ANN. 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [5]:
# In[5]:
# Feature Scaling to standardize the range of independent variables or features of data (normalize).
# we want to have standard normally distributed data and dont want to have one independent var dominating another one.
# Also due to intensive calculations and parallel execution, scaling eases these calculations.

# If one of the features has a broad range of values, the distance will be governed by
# this particular feature. Therefore, the range of all features should be normalized so 
# that each feature contributes approximately proportionately to the final distance.

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

In [6]:
'''(you have to use the same two parameters μ and σ (values) that you used for centering the training set)
sklearn's transform's fit() just calculates the parameters (e.g. μ and σ)
and saves them as an internal objects state. 
Afterwards, you can call its transform() method to apply the transformation to a particular set of examples.
'''
X_train = sc.fit_transform(X_train) # Fit to data, then transform it.
X_test = sc.transform(X_test) # Perform standardization by centering and scaling


In [7]:
# In[6]:
# Intitialize the ANN

# ===========================
# Make the ANN
# ===========================

# Importing the Keras libraries and packages
import keras
from keras.models import Sequential # to initialize ANN
from keras.layers import Dense # to create layers in ANN

# Initialising the ANN (we will define it as a sequence of layers or you can define a graph)
classifier = Sequential()
# Add the input layer AND the first hidden layer. (for initial layer specify the input nodes since we have no prev layer)
classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu', input_dim = 23))

# Add the second hidden layer (use prev layer as input nodes)
classifier.add(Dense(units = 12, kernel_initializer = 'uniform', activation = 'relu'))

# Add the output layer (NOTE: if 3 encoded categories for dependent variable need 3 nodes and softmax activator func)
# choose sigmoid just like in logistic regression
classifier.add(Dense(units = 1, kernel_initializer = 'uniform', activation = 'sigmoid'))
classifier.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 12)                288       
_________________________________________________________________
dense_1 (Dense)              (None, 12)                156       
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 13        
Total params: 457
Trainable params: 457
Non-trainable params: 0
_________________________________________________________________


In [8]:
'''
1. Dense() function will randomly initialize the weights to small numbers close to 0 (but not 0)
2. place each feature in one input node e.g: 11 input vars => 11 input nodes
3. farward propagation pass inputs from left to right where the neuron will be activated in way
   where impact of each neurons activation is limited by the weights. (sum allweights, apply activation function)
   using rectifier func for input layer and sigmoid for outer layer(to get probabilities of customer leaving or staying) 
4. compare predicted result to actual
5. back propagation from right to left. update the weights according to how much they are responsible for he error
   The learnign rate decides by how much we update the weights
6. repeat steps 1-5 and update weights after each observation (row) (reinforcement learning)
   Or: repeat steps 1-5 and updatee the weights only after a batch of observations (Batch learning)
7. When the whole training (all rows of data) passed through the ANN that makes an epoch. Redo more epochs
'''
# use stochastic gradient decscent


#Choose Number of nodes in hidden layer: as average number of nodes in hidden and output layer
#Or experiment with parameter tuning, ross validation techniques
# (23 + 1)/2 for hidden layer

# initialize weights randomly close to 0: kernel_initializer = 'uniform'

# using rectifier activation func for input layter and sigmoid for outer layer(to get probabilities) 


'\n1. Dense() function will randomly initialize the weights to small numbers close to 0 (but not 0)\n2. place each feature in one input node e.g: 11 input vars => 11 input nodes\n3. farward propagation pass inputs from left to right where the neuron will be activated in way\n   where impact of each neurons activation is limited by the weights. (sum allweights, apply activation function)\n   using rectifier func for input layer and sigmoid for outer layer(to get probabilities of customer leaving or staying) \n4. compare predicted result to actual\n5. back propagation from right to left. update the weights according to how much they are responsible for he error\n   The learnign rate decides by how much we update the weights\n6. repeat steps 1-5 and update weights after each observation (row) (reinforcement learning)\n   Or: repeat steps 1-5 and updatee the weights only after a batch of observations (Batch learning)\n7. When the whole training (all rows of data) passed through the ANN tha

In [9]:
# In[7]:
# Compile the ANN (apply Stochastic gradient descent for back propagation)
# Stochastic gradient descent algorithm = adam. 
# loss function within adam alg, based on loss function that we need to optimize to find optimal weights 

# (for simple linear regression loss func is sum of squared errors. 
# in ML(perceptron NN using sigmoid activation function you obtain a logistic regression model):
# looking into stochastic gradient descent loss func is NOT sum of squared errors but is a 
# logarithmic function called logarithmic loss )

# so use binary logarithmic loss func b/c (binary_entropy = dependent var has binary outcome, if i categories = categorical_crossentropy)
# criteria to evaluate our model metrics = ['accuracy'] (after weights updated, algo uses accuracy criterion to improve models performance) (when we fit accuracy will increase little by litle until reach top accuracy since we chose accuracy metric)
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])


In [10]:
# In[8]:
# Fit the ANN to the Training set (experimment with batch size and epochs)
# batch_size = 10, epochs = 100
# loss: 0.4234 - acc: 0.8193
hist=classifier.fit(X_train, y_train, batch_size = 10, epochs = 100,validation_split=0.2)


Epoch 1/100
1920/1920 [==============================] - 4s 2ms/step - loss: 0.4880 - accuracy: 0.7919 - val_loss: 0.4667 - val_accuracy: 0.8104
Epoch 2/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4512 - accuracy: 0.8170 - val_loss: 0.4551 - val_accuracy: 0.8108
Epoch 3/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4464 - accuracy: 0.8186 - val_loss: 0.4529 - val_accuracy: 0.8096
Epoch 4/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4435 - accuracy: 0.8196 - val_loss: 0.4521 - val_accuracy: 0.8112
Epoch 5/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4419 - accuracy: 0.8184 - val_loss: 0.4472 - val_accuracy: 0.8146
Epoch 6/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4411 - accuracy: 0.8190 - val_loss: 0.4493 - val_accuracy: 0.8131
Epoch 7/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4406 - accuracy: 0.8195 - val_loss: 0.4500 - val_ac

1920/1920 [==============================] - 3s 2ms/step - loss: 0.4253 - accuracy: 0.8221 - val_loss: 0.4467 - val_accuracy: 0.8096
Epoch 58/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4251 - accuracy: 0.8241 - val_loss: 0.4446 - val_accuracy: 0.8133
Epoch 59/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4250 - accuracy: 0.8231 - val_loss: 0.4436 - val_accuracy: 0.8104
Epoch 60/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4252 - accuracy: 0.8219 - val_loss: 0.4471 - val_accuracy: 0.8112
Epoch 61/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4253 - accuracy: 0.8231 - val_loss: 0.4435 - val_accuracy: 0.8110
Epoch 62/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4247 - accuracy: 0.8229 - val_loss: 0.4431 - val_accuracy: 0.8110
Epoch 63/100
1920/1920 [==============================] - 3s 2ms/step - loss: 0.4247 - accuracy: 0.8222 - val_loss: 0.4443 - val_accuracy

In [11]:
val_accu=hist.history['val_accuracy'][-1]
accu=hist.history['accuracy'][-1]
accu,val_accu

(0.8227083086967468, 0.8100000023841858)

In [12]:
classifier.evaluate(X_test,y_test)

188/188 [==============================] - 0s 2ms/step - loss: 0.4355 - accuracy: 0.8243


[0.4355347752571106, 0.8243333101272583]

In [27]:
predict=classifier.predict(X_test)
predict

array([[0.66558254],
       [0.29539126],
       [0.22784236],
       ...,
       [0.12836903],
       [0.73417   ],
       [0.2619006 ]], dtype=float32)

In [28]:
predict[predict>0.5]=1
predict[predict<0.5]=0

In [29]:
predict

array([[1.],
       [0.],
       [0.],
       ...,
       [0.],
       [1.],
       [0.]], dtype=float32)

In [31]:
y_test=y_test.reshape(6000,1)
np.sum(y_test==predict)/predict.shape[0]

0.8243333333333334

In [32]:
# Make the Confusion Matrix to evaluate
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, predict)

In [33]:
cm

array([[4451,  252],
       [ 802,  495]], dtype=int64)